<a href="https://colab.research.google.com/github/anuraglahon16/AI-Engineering/blob/main/Copy_of_OpenAI_Assistants_Building_Agentic_RAG_with_Function_Calling_API_and_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [ ]:
!pip install -qU openai

In [ ]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

OpenAI API Key:··········


## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [ ]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [ ]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "Anurag" # @param {type: "string"}
instructions = "Be polite " # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [ ]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [ ]:
assistant

Assistant(id='asst_nekvWOJOTZ73qiG09U5tUaDP', created_at=1709569552, description=None, file_ids=[], instructions='Be polite ', metadata={}, model='gpt-3.5-turbo', name='Anurag', object='assistant', tools=[])

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [ ]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [ ]:
thread

Thread(id='thread_j42CumceGGSQ0ZHnpeY93D5X', created_at=1709569552, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How old are you?"
)

Again, let's examine our `message` object!

In [ ]:
message

ThreadMessage(id='msg_sUez0YZ1sd7g1yJvXCkMpasO', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='How old are you?'), type='text')], created_at=1709569553, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_j42CumceGGSQ0ZHnpeY93D5X')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [ ]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "Break down complex tasks into a sequence of simpler prompts in an interactive conversation INSTRUCTIONS" # @param {type: "string"}

Let's run our Thread!

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [ ]:
run

Run(id='run_a1Yp7KRWBwf36yY9jjoN4dfr', assistant_id='asst_nekvWOJOTZ73qiG09U5tUaDP', cancelled_at=None, completed_at=None, created_at=1709569553, expires_at=1709570153, failed_at=None, file_ids=[], instructions='Be polite  Break down complex tasks into a sequence of simpler prompts in an interactive conversation INSTRUCTIONS', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_j42CumceGGSQ0ZHnpeY93D5X', tools=[], usage=None)

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [ ]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [ ]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [ ]:
messages.data[0]

ThreadMessage(id='msg_AhH2J1NmwQbr1RFSeenZmLym', assistant_id='asst_nekvWOJOTZ73qiG09U5tUaDP', content=[MessageContentText(text=Text(annotations=[], value="I'm just a computer program, so I don't have an age in the way humans do. How can I assist you today?"), type='text')], created_at=1709569554, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_a1Yp7KRWBwf36yY9jjoN4dfr', thread_id='thread_j42CumceGGSQ0ZHnpeY93D5X')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [ ]:
!wget https://www.gutenberg.org/files/84/84-h/84-h.htm -o frankenstein.html

Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [ ]:
file_reference = client.files.create(
  file=open("frankenstein.html", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [ ]:
file_reference

FileObject(id='file-vzfWc5UETezsvE6P29HnOSdV', bytes=1204, created_at=1709569555, filename='frankenstein.html', object='file', purpose='assistants', status='processed', status_details=None)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [ ]:
assistant = client.beta.assistants.create(
  name=name + "+ Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [ ]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is the first words Victor Frankenstein speaks?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress


Let's look at the final result!

In [ ]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_lwMQz0roAWgvjxhA0E9SOr6r', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is the first words Victor Frankenstein speaks?'), type='text')], created_at=1709569558, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_rSEhezCFv0NuZxlRVmguMvaZ')], object='list', first_id='msg_lwMQz0roAWgvjxhA0E9SOr6r', last_id='msg_lwMQz0roAWgvjxhA0E9SOr6r', has_more=False)

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [ ]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [ ]:
assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [ ]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [ ]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_ctfGFh1C8Mf3khkKwtAUfy3B'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeToolCall(id='call_iyqgrlVLnf3MGSqoisEJYzha', code_interpreter=CodeInterpreter(input='import subprocess\r\n\r\n# Use the `file` command to determine the file type\r\noutput = subprocess.check_output([\'file\', filename]).decode()\r\nfile_type = output.split(":")[-1].strip()\r\n\r\nfile_type', outputs=[CodeInterpreterOutputLogs(logs="'Unicode text, UTF-8 text'", type='logs')]), type='code_interpreter')], type='tool_calls')
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_3jaRGxDFZBAJ55v26uHFMKyk'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeToolCall(id='call_xcrTnRmYtdMCSTU2lzVnVgGj', code_interpreter=CodeInterpreter(input="import mimetypes\r\n\r\n# Get the file type\r\nfilename = '/mnt/data/file-vzfWc5UETezsvE6P29HnOSdV'\r\nfile_type, _ = mimetypes.guess_type(filename)\r\n\r\nfi

In [ ]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_ctfGFh1C8Mf3khkKwtAUfy3B', assistant_id='asst_MrsjADqNKffVZQOhzMfGcuf9', content=[MessageContentText(text=Text(annotations=[], value='The file you uploaded appears to be a "Unicode text, UTF-8 text" file. If you need further assistance with this file or have any other questions, feel free to let me know!'), type='text')], created_at=1709570775, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_8nZPUcfqo0t7THOyzCCCYDyS', thread_id='thread_PZG6cCKdxLWX3AFgWO5sQEpU'), ThreadMessage(id='msg_3jaRGxDFZBAJ55v26uHFMKyk', assistant_id='asst_MrsjADqNKffVZQOhzMfGcuf9', content=[MessageContentText(text=Text(annotations=[], value='It seems that I was unable to determine the file type using the alternative method. Let me try another approach to identify the file type.'), type='text')], created_at=1709570773, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_8nZPUcfqo0t7THOyzCCCYDyS

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [ ]:
!pip install -qU duckduckgo_search

In [ ]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [ ]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

'Winnipeg Jets forward Adam Lowry has been named as captain of the team. The Jets made the announcement on Tuesday morning, saying that Lowry is the third captain since the Jets returned in 2011.\nThe official 2023 - 2024 roster of the Winnipeg Jets, including position, height, weight, date of birth, age, and birth place.\nLowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ...'

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [ ]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

####❓ Question

Why does the description key-value pair matter?


The description key-value pair is essential because it clearly explains a function's purpose and usage, guiding developers and users in how to properly use it. It improves clarity, documentation, and usability, making interactions with the function more straightforward and error-free.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [ ]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [ ]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

####❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

The use_assistant helper function sends a query to an OpenAI Assistant by creating a new conversation thread, or adding to an existing one if a thread ID is provided. It effectively initiates or continues a dialogue with the assistant by posting the user's question within this thread, allowing for an organized conversation flow.

In [ ]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_YvfJg5cgKQYlTDnyNR653JYc', 'output': 'Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nWinnipeg Jets Captains. Team Names: Winnipeg Jets, Atlanta Thrashers. Seasons: 24 (1999-00 to 2023-24) NHL Playoff Appearances: 7. NHL Championships: 0 (0 Stanley Cup) ... Current Summary/Standings, Current Schedule/Results, Current Leaders, Current Stats. 2023-24, ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nThe official 2023 - 2024 roster of the Winnipeg Jets, including position, height, weight, date of birth, age, and birt

'thread_45vU1KHXgG9bZ7IYGl4cvTsS'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [ ]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [ ]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_ZZYiu33dtJvsRpQ2uEsywxXJ', 'output': "Winnipeg Jets Captains. Team Names: Winnipeg Jets, Atlanta Thrashers. Seasons: 24 (1999-00 to 2023-24) NHL Playoff Appearances: 7. NHL Championships: 0 (0 Stanley Cup) ... Current Summary/Standings, Current Schedule/Results, Current Leaders, Current Stats. 2023-24, ...\nThe Winnipeg Jets have a new leader, one year after stripping the C from Blake Wheeler and deciding to play without a captain. Adam Lowry, who has been a Jet since 2011 when he was drafted 67th ...\nLowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, aft

'thread_DreCEz81pk9jy8zaeYGtcOR6'

In [ ]:
use_assistant("Who is the author of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: The author of the supplied file is Mary Shelley. The file contains the text of the novel "Frankenstein" by Mary Shelley【5†source】.
user: Who is the author of the supplied file?


'thread_kGxm3SWwhFtEb1lF51GQ1brJ'

In [ ]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_QYKaU6NkcLEcBY1yDZpfkPR7', 'output': 'Here are some advantages to use du command to check file size in Linux. The du command can show the size of a file in human-readable format, which makes it easier to interpret the results.; The du command can be used to check the size of multiple files at once.; The du command can be used to check the size of directory.; The du command is faster than the ls command when checking the size of ...\nThe du command can be used to check the size of files, directories, and the total disk space used by the current directory and subdirectories. Run "du -h" to see a list of files and folders in a human-readable format. When you use the Linux du command, you obtain both the actual disk usage and the true size of a file or directory.\nBy default, the block size in most Linux system is 4096 Bytes or 4 KB. A directory in Li

'thread_eRH6acmQ3nSLotEOnlIooYnb'

####❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?


The decision on which tool to use is determined by the OpenAI Assistant's internal logic, considering the query's nature and the available tools' capabilities to choose the best response method.

### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [ ]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [ ]:
query = "How many bytes is the provided file?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_JWDAfFDNG9KA6Jq0TM1cQ0EP', 'output': 'Here are some advantages to use du command to check file size in Linux. The du command can show the size of a file in human-readable format, which makes it easier to interpret the results.; The du command can be used to check the size of multiple files at once.; The du command can be used to check the size of directory.; The du command is faster than the ls command when checking the size of ...\nThe du command can be used to check the size of files, directories, and the total disk space used by the current directory and subdirectories. Run "du -h" to see a list of files and folders in a human-readable format. When you use the Linux du command, you obtain both the actual disk usage and the true size of a file or directory.\nA directory in Linux is simply a file with the information about the memory location of 

Now we can observe JSON mode in action!

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [ ]:
completed_flag

{'completed': True}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [ ]:
### YOUR CODE HERE

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [ ]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)